# HyperCLOVA X 캐릭터챗봇 튜닝용 데이터셋 만들기
📅 2024-07-06   
🧑🏻‍💻 외계공룡
    
# 🚀 사용방법
1. colab(https://colab.google/) 접속
2. `hcx_character_datasets_generator.ipynb` 불러오기
3. `datasets.csv` 파일 업로드
4. API 정보 입력
5. 프롬프트 변경
6. 전체 코드 실행

In [1]:
!python --version # 파이썬 버전확인

Python 3.10.12


### API 정보 입력
- ❗하단에 CLOVA Studio 테스트 앱 API 정보 입력

In [12]:
X_NCP_CLOVASTUDIO_API_KEY='X-NCP-CLOVASTUDIO-API-KEY'
X_NCP_APIGW_API_KEY='X-NCP-APIGW-API-KEY'
X_NCP_CLOVASTUDIO_REQUEST_ID='X-NCP-CLOVASTUDIO-REQUEST-ID'

### 캐릭터 답변생성 프롬프트
- 🪄 원하는 캐릭터 답변생성 프롬프트로 변경    
- 하단은 귀멸의칼날 '카마도 탄지로' 캐릭터 예시


In [34]:
prompt = '''
1. 당신은 일본애니메이션인 귀멸의칼날에 등장하는 주인공 카마도 탄지로 입니다.
2. 카마도 탄지로가 되어서 사용자와 대화를 진행합니다.
3. 아래의 ###아이덴티티###를 참고하세요.
4. ###대화 패턴###을 참고해서 카마도 탄지로처럼 질문에 대해 답변해주세요.

###아이덴티티###
https://namu.wiki/w/%EC%B9%B4%EB%A7%88%EB%8F%84%20%ED%83%84%EC%A7%80%EB%A1%9C

###대화 패턴###
- 카마도 탄지로는 본인보다 나이가 많으면 ~씨라는 존칭을 붙인다.
- 사용자에게는 반말로 답변한다.
- 등장인물들을 자주 언급한다.
- 카마도 탄지로는 과거회상을 자주한다.
- 답변을 길게한다.
'''

# 실행 코드
- row 수 만큼 반복문 실행됨
- 💸 과금 주의

In [48]:
import json
import requests
import csv

class CompletionGenerator:
    host='https://clovastudio.stream.ntruss.com'
    def __init__(self):
        pass

    def execute(self, completion_request):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': X_NCP_CLOVASTUDIO_API_KEY,
            'X-NCP-APIGW-API-KEY': X_NCP_APIGW_API_KEY,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': X_NCP_CLOVASTUDIO_REQUEST_ID,
            'Content-Type': 'application/json; charset=utf-8',
        }

        response = requests.post(self.host + '/testapp/v1/chat-completions/HCX-003',headers=headers, json=completion_request, stream=False)
        json_data = json.loads(response.text)
        return json_data['result']['message']['content']


if __name__ == '__main__':

    with open('datasets.csv', 'r') as file:
        reader = csv.reader(file)
        rows = list(reader)

    completion_generator = CompletionGenerator()

    for row in rows:
        if not row[3]:
            request_body = {
                'messages': [
                    {"role":"system","content":prompt},
                    {"role":"user","content":row[2]}
                ],
                'topP': 0.8,
                'topK': 0,
                'maxTokens': 500,
                'temperature': 0.8,
                'repeatPenalty': 5.0,
                'stopBefore': [],
                'includeAiFilters': True,
                'seed': 0
            }

            row[3] = completion_generator.execute(request_body)
            print(row[3])

    with open('datasets.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)